# Biomaker CA: create a world with rain

In this colab we implement WATER and WATER_GENERATOR types. Rain falls from the sky and it is the only source of earth nutrients in the environment.

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Imports

In [ ]:
#@title install selforg package
# install the package locally
!pip install --upgrade -e git+https://github.com/google-research/self-organising-systems.git#egg=self_organising_systems&subdirectory=biomakerca
# activate the locally installed package (otherwise a runtime restart is required)
import pkg_resources
pkg_resources.get_distribution("self_organising_systems").activate()

In [ ]:
#@title imports & notebook utilities
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.mutators import CrossOverSexualMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom, tile2d, add_text_to_img, imshow
from self_organising_systems.biomakerca.custom_ipython_display import display
# imports for creating new types
from self_organising_systems.biomakerca.utils import conditional_update
from self_organising_systems.biomakerca.utils import arrayContains
from self_organising_systems.biomakerca import cells_logic
from self_organising_systems.biomakerca import env_logic

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time
import matplotlib.pyplot as plt

import tqdm
import mediapy as media
from functools import partial


def pad_text(img, text):
  font = cv2.FONT_HERSHEY_SIMPLEX
  orgin = (5, 15)
  fontScale = 0.5
  color = (0, 0, 0)
  thickness = 1

  # ensure to preserve even size (assumes the input size was even.
  new_h = img.shape[0]//15
  new_h = new_h if new_h % 2 == 0  else new_h + 1
  img = np.concatenate([np.ones([new_h, img.shape[1], img.shape[2]]), img], 0)
  img = cv2.putText(img, text, orgin, font, fontScale, color, thickness, cv2.LINE_AA)
  return img

# Create a new configuration and types

In [ ]:
### New EnvTypeDef

RAINY_MATERIALS = evm.DEFAULT_MATERIALS + ["WATER", "WATER_GENERATOR"]
RAINY_TYPE_COLOR_DICT = dict(
    evm.DEFAULT_TYPE_COLOR_DICT,
    **{"WATER": jp.array([0., 0., 0.545]),  # RGB: 0,0,139
       "WATER_GENERATOR": jp.array([0.545, 0., 0.545])})  # RGB: 139,0,139
RAINY_STRUCTURE_DECAY_MATS_DICT = dict(
    evm.DEFAULT_STRUCTURE_DECAY_MATS_DICT,
    **{"WATER": -1, "WATER_GENERATOR": -1})


class RainyTypeDef(evm.DefaultTypeDef):
  """EnvTypeDef for Rainy.

  WATER is a gravity mat.
  WATER and WATER_GENERATOR both propagate air nutrients.
  """

  def __init__(self):
    super().__init__(
        RAINY_MATERIALS, evm.DEFAULT_AGENT_TYPES,
        RAINY_STRUCTURE_DECAY_MATS_DICT,
        evm.DEFAULT_DISSIPATION_RATE_PER_SPEC_DICT,
        RAINY_TYPE_COLOR_DICT)
    types = self.types

    # Now add material specific properties.
    # Immutable only propagates structural integrity.
    self.immutable_creates_nutrients = False
    # WATER is a gravity_mat
    self.gravity_mats = jp.concatenate([
        self.gravity_mats,
        jp.array([types.WATER], dtype=jp.int32)], 0)
    # WATER and WATER_GENERATOR propagate air nutrients.
    self.propagate_air_nutrients_mats = jp.concatenate([
        self.propagate_air_nutrients_mats,
        jp.array([types.WATER, types.WATER_GENERATOR], dtype=jp.int32)], 0)


### New ExclusiveOps

WATER_NUTRIENT_INC_PERC = 0.1

def water_cell_op(key, perc, config):
  """Create the exclusive function of WATER cells.

  First, WATER checks if there is EARTH directly under it. If there is, it gets
  absorbed by it and it becomes AIR.

  WATER has a falling-sand property.
  """
  neigh_type, neigh_state, neigh_id = perc
  etd = config.etd

  # Create the output (and modify it based on conditions)
  t_upd_mask = env_logic.EMPTY_UPD_MASK
  a_upd_mask = env_logic.EMPTY_UPD_MASK
  t_upd_type = env_logic.EMPTY_UPD_TYPE
  a_upd_type = env_logic.EMPTY_UPD_TYPE
  t_upd_state = env_logic.make_empty_upd_state(config)
  a_upd_state = env_logic.make_empty_upd_state(config)
  t_upd_id = env_logic.EMPTY_UPD_ID
  a_upd_id = env_logic.EMPTY_UPD_ID


  ## water is absorbed if there is EARTH below.
  # extra: Roots also absorb nutrients!
  #absorbing_types = jp.array([etd.types.EARTH, etd.types.AGENT_ROOT])

  down_idx = 7
  is_earth_below_i = (neigh_type[down_idx] == etd.types.EARTH).astype(jp.int32)
  #is_earth_below_i = arrayContains(absorbing_types, neigh_type[down_idx])
  # update the output accordingly.
  t_upd_mask = t_upd_mask.at[down_idx].set(is_earth_below_i)
  a_upd_mask = a_upd_mask.at[down_idx].set(is_earth_below_i)
  # the type of the target remains the same.
  t_upd_type = conditional_update(
      t_upd_type, down_idx, neigh_type[down_idx], is_earth_below_i)
  # water becomes air.
  a_upd_type = conditional_update(
      a_upd_type, down_idx, etd.types.AIR, is_earth_below_i)
  # target earth_nutrients are increased.
  earth_cap = config.material_nutrient_cap[evm.EARTH_NUTRIENT_RPOS]
  target_new_state = neigh_state[down_idx].at[evm.EN_ST + evm.EARTH_NUTRIENT_RPOS].set(
      jp.minimum(earth_cap, neigh_state[evm.EN_ST + down_idx,evm.EARTH_NUTRIENT_RPOS] +
                 earth_cap*WATER_NUTRIENT_INC_PERC))
  t_upd_state = conditional_update(
      t_upd_state, down_idx, target_new_state, is_earth_below_i)
  # actor state is unchanged (so that air nutrients are preserved).
  a_upd_state = conditional_update(
      a_upd_state, down_idx, neigh_state[4], is_earth_below_i)
  # the target id is the same (relevant for AGENT types).
  t_upd_id = conditional_update(
      t_upd_id, down_idx, neigh_id[down_idx], is_earth_below_i)
  # the actor id is always 0 so it doesn't matter.

  done_i = is_earth_below_i
  # Then, check if you can fall.
  # for now, you can't fall out of bounds.
  # if you can fall, do nothing. Gravity will take care of it.
  can_fall_i = (1 - done_i) * jp.logical_and(
      neigh_type[7] != etd.types.OUT_OF_BOUNDS,
      arrayContains(etd.intangible_mats, neigh_type[7])
  )

  done_i = (done_i + can_fall_i).clip(max=1)

  # Else, check if you can fall on a random side.
  # both the side and below need to be free.
  # if yes, do that.
  key, ku = jr.split(key)
  side_idx = jr.choice(ku, jp.array([3, 5]))
  down_side_idx = side_idx + 3

  can_fall_to_side_i = (1 - done_i) * (
      (neigh_type[side_idx] != etd.types.OUT_OF_BOUNDS)
      & (neigh_type[down_side_idx] != etd.types.OUT_OF_BOUNDS)
      & arrayContains(etd.intangible_mats, neigh_type[side_idx])
      & arrayContains(etd.intangible_mats, neigh_type[down_side_idx])
  )

  # update the outputs if true.
  t_upd_mask = conditional_update(t_upd_mask, side_idx, 1.0, can_fall_to_side_i)
  a_upd_mask = conditional_update(a_upd_mask, side_idx, 1.0, can_fall_to_side_i)
  # switch the types, states and ids
  t_upd_type = conditional_update(
      t_upd_type, side_idx, neigh_type[4], can_fall_to_side_i)
  a_upd_type = conditional_update(
      a_upd_type, side_idx, neigh_type[side_idx], can_fall_to_side_i)
  t_upd_state = conditional_update(
      t_upd_state, side_idx, neigh_state[4], can_fall_to_side_i)
  a_upd_state = conditional_update(
      a_upd_state, side_idx, neigh_state[side_idx], can_fall_to_side_i)
  t_upd_id = conditional_update(
      t_upd_id, side_idx, neigh_id[4], can_fall_to_side_i.astype(jp.uint32))
  a_upd_id = conditional_update(
      a_upd_id, side_idx, neigh_id[side_idx],
      can_fall_to_side_i.astype(jp.uint32))

  return env_logic.ExclusiveOp(
      env_logic.UpdateOp(t_upd_mask, t_upd_type, t_upd_state, t_upd_id),
      env_logic.UpdateOp(a_upd_mask, a_upd_type, a_upd_state, a_upd_id),
  )

GENERATE_WATER_CHANCE = 0.025

def water_generator_cell_op(key, perc, config):
  """Create the exclusive function of WATER_GENERATOR cells.

  WATER_GENERATORS generate WATER below them.
  This happens only if the type below is intangible, and with a certain chance.
  This operation preserves air nutrients.

  This operation doesn't affect the water generator.
  """
  neigh_type, neigh_state, neigh_id = perc
  etd = config.etd

  down_idx = 7
  key, ku = jr.split(key)
  take_action_i = (arrayContains(etd.intangible_mats, neigh_type[down_idx]) &
                   (jr.uniform(ku) < GENERATE_WATER_CHANCE)).astype(jp.int32)

  t_upd_mask = env_logic.EMPTY_UPD_MASK.at[down_idx].set(take_action_i)
  a_upd_mask = env_logic.EMPTY_UPD_MASK
  t_upd_type = conditional_update(
      env_logic.EMPTY_UPD_TYPE, down_idx, etd.types.WATER, take_action_i)
  a_upd_type = env_logic.EMPTY_UPD_TYPE
  t_upd_state = conditional_update(
      env_logic.make_empty_upd_state(config), down_idx, neigh_state[down_idx],
      take_action_i)
  a_upd_state = env_logic.make_empty_upd_state(config)
  t_upd_id = env_logic.EMPTY_UPD_ID
  a_upd_id = env_logic.EMPTY_UPD_ID

  return env_logic.ExclusiveOp(
      env_logic.UpdateOp(t_upd_mask, t_upd_type, t_upd_state, t_upd_id),
      env_logic.UpdateOp(a_upd_mask, a_upd_type, a_upd_state, a_upd_id),
  )

# Create the exclusive fs
def make_rainy_excl_fs(etd):
  return (
      (etd.types.AIR, cells_logic.air_cell_op),
      (etd.types.EARTH, cells_logic.earth_cell_op),
      (etd.types.WATER, water_cell_op),
      (etd.types.WATER_GENERATOR, water_generator_cell_op),
      )

### Rainy config
def get_rainy_config():
  """Return a new Rainy config.

  This config is quite trivial. It actually is taken from 'persistence'.
  """
  return evm.get_env_and_config(
      "persistence", width_type="petri", h=10, etd=RainyTypeDef()).config

### Rainy environment

def create_rainy_env(h, w, config):
  """Create the Rainy environment.
  """
  env = evm.create_multiseed_environment(h, w, config)
  # If you wanted an environment with a single initial seed instead.
  #env = evm.create_default_environment(config, h, w)
  #env = evm.place_seed(env, w//2, config)
  env = evm.update_env_type_grid(env, env.type_grid.at[1].set(
      config.etd.types.WATER_GENERATOR))
  return env


## Select the configuration, the agent logic and the mutator

Set soil_unbalance_limit to 0 to reproduce the original environment. Set it to 1/3 for having self-balancing environments (recommended).

In [ ]:
ec_id = "persistence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "wide" #@param ['wide', 'landscape', 'square', 'petri', '10x', '20x']
soil_unbalance_limit = 1/3 #@param [0, "1/3"] {type:"raw"}

h = 72
if env_width_type == "10x":
  env_width_type = h * 10
if env_width_type == "20x":
  env_width_type = h * 20
else:
  env_width_type = evm.infer_width(h, env_width_type)

config = get_rainy_config()


etd = config.etd
# Create the exclusive fs
excl_fs = make_rainy_excl_fs(etd)

st_env = create_rainy_env(h, env_width_type, config)

config.soil_unbalance_limit = soil_unbalance_limit
reproduction_type = "both" #@param ['both', 'asexual', 'sexual']
does_sex_matter = True #@param ['False', 'True'] {type:"raw"}

enable_asexual_reproduction = reproduction_type != "sexual"
enable_sexual_reproduction = reproduction_type != "asexual"

sex_sensitivity = 1000 #@param [1, 10, 100, 1000] {type:"raw"}

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal",
                              make_asexual_flowers_likely=enable_asexual_reproduction,
                              make_sexual_flowers_likely=enable_sexual_reproduction,
                              init_noise=0.001, sex_sensitivity=sex_sensitivity)

n_sparse_max = 2**13 #@param ['None', '2**13', '2**12', '2**11', '2**10'] {type:"raw"}

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))
sexual_mutator = CrossOverSexualMutator(mutator, n_frequencies=64)

exp_id = "{}_sm^{}_sens^{}_w^{}".format(reproduction_type, does_sex_matter, sex_sensitivity, env_width_type)
print(exp_id)

## Optionally, modify the config for custom configurations

In [ ]:
print("Current config:")
print('\n'.join("%s: %s" % item for item in vars(config).items()))

In [ ]:
## Examples for modifying the config
## Uncomment relevant lines or do like them.

## Regardless, to trigger the recomputation of step_env and similar,
## config needs to be a new object! So, first, we create a new copy.
import copy
config = copy.copy(config)

## Change simple isolated parameters (most of them)
# config.struct_integrity_cap = 100
# config.max_lifetime = 500
## Vectors can be modified either by writing new vectors:
# config.dissipation_per_step = jp.array([0.02, 0.02])
## Or by multiplying previous values. Note that they are immutable!
# config.dissipation_per_step = config.dissipation_per_step * 2

## agent_state_size is trickier, because it influences env_state_size.
## So you can either create a new config:
## Note that you would have to insert all values that you don't want to take
## default initializations.
# config = evm.EnvConfig(agent_state_size=4)
## Or you can just modify env_state_size as well.
## (env_state_size = agent_state_size + 4) for now.
# config.agent_state_size = 4
# config.env_state_size = config.agent_state_size + 4


## Perform a simulation

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:
# create auxiliary frames that show interesting counters.

def make_aux_frame(n_asex, n_sex):
  img = np.ones([130 if reproduction_type == "both" else 65, 550, 3])
  yorigin = 50
  if enable_asexual_reproduction:
    img = add_text_to_img(
        img, "Asexual reproductions: {}".format(n_asex),
        origin=(20, yorigin), color="black")
    yorigin = 100
  if enable_sexual_reproduction:
    img = add_text_to_img(
        img, "Sexual reproductions:  {}".format(n_sex),
        origin=(20, yorigin), color="black")
  return img

imshow(make_aux_frame(100000, 212122))

def make_nsexes_frame(n_sexes):
  img = np.ones([65, 300, 3])
  yorigin = 50
  img = add_text_to_img(
      img, "Num sexes: {}".format(n_sexes),
      origin=(20, yorigin), color="black")
  return img

imshow(make_nsexes_frame(200))

In [ ]:

@partial(jit, static_argnames=["config", "n_max_programs"])
def get_alive_programs_mask(env, config, n_max_programs):
  aid_flat = env.agent_id_grid.flatten()
  is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
  n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=n_max_programs)

  has_alive = n_alive_per_id > 0
  return has_alive

@jit
def get_alive_and_sexes(env, programs):
  has_alive = get_alive_programs_mask(env, config, N_MAX_PROGRAMS)
  all_sexes = vmap(agent_logic.get_sex)(mutator.split_params(programs)[0])
  return has_alive, all_sexes

def get_num_sexes(env, programs):
  has_alive, all_sexes = get_alive_and_sexes(env, programs)
  has_alive = np.array(has_alive)
  all_sexes = np.array(all_sexes)

  sexes = all_sexes[has_alive]
  return len(np.unique(sexes))

def run_env(
    key, programs, env, n_steps, step_f,
    curr_asexual_repr = 0, curr_sexual_repr = 0,
    zoom_sz=12,
    steps_per_frame=2, when_to_double_speed=[100, 500, 1000, 2000, 5000]):

  fps = 20
  def make_frame(env):
    return zoom(evm.grab_image_from_env(
        env, config, id_color_intensity=0.3),zoom_sz)

  frame = make_frame(env)

  # remember that the metrics are per step, right now, and that are 'inbetween'
  # frames, at best.
  n_asexual_repr_log = []
  n_sexual_repr_log = []
  n_sexes_log = []

  aux_frames = [make_aux_frame(0, 0)]
  num_sexes_frames = [make_nsexes_frame(get_num_sexes(env, programs))]

  out_file = "video.mp4"
  with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps, crf=18
                        ) as video:
    video.add_image(frame)
    for i in tqdm.trange(n_steps):
      if i in when_to_double_speed:
        steps_per_frame *= 2

      key, ku = jr.split(key)
      (env, programs), metrics = step_f(ku, env, programs=programs)

      if enable_asexual_reproduction:
        step_asex_repr = int(metrics["asexual_reproduction"][0].sum())
        n_asexual_repr_log.append(step_asex_repr)
        curr_asexual_repr += step_asex_repr
      if enable_sexual_reproduction:
        step_sexual_repr = int(metrics["sexual_reproduction"][0].sum())
        n_sexual_repr_log.append(step_sexual_repr)
        curr_sexual_repr += step_sexual_repr

      # get sexes alive
      num_sexes = get_num_sexes(env, programs)
      n_sexes_log.append(num_sexes)
      if i % steps_per_frame == 0:
        video.add_image(make_frame(env))
        aux_frames.append(make_aux_frame(curr_asexual_repr, curr_sexual_repr))
        num_sexes_frames.append(make_nsexes_frame(num_sexes))


  media.show_video(media.read_video(out_file))
  media.show_video(aux_frames, fps=fps)
  media.show_video(num_sexes_frames, fps=fps)
  ret_metrics = {'n_asexual_repr_log': n_asexual_repr_log,
                 'n_sexual_repr_log': n_sexual_repr_log,
                 'n_sexes_log': n_sexes_log,
                 'curr_asexual_repr': curr_asexual_repr,
                 'curr_sexual_repr': curr_sexual_repr}
  return programs, env, ret_metrics

In [ ]:
key = jr.PRNGKey(42)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 512

# for 20x environments, you need shorter videos.
n_steps = 10000

# on what FRAME to double speed.
when_to_double_speed = [100, 200, 300, 400, 500]
# on what FRAME to reset speed.
when_to_reset_speed = []
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 4

ku, key = jr.split(key)
programs = vmap(agent_logic.initialize)(jr.split(ku, N_MAX_PROGRAMS))
ku, key = jr.split(key)
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

step_f = partial(step_env, config=config, agent_logic=agent_logic, do_reproduction=True,
          enable_asexual_reproduction=enable_asexual_reproduction,
          enable_sexual_reproduction=enable_sexual_reproduction,
          does_sex_matter=does_sex_matter,
          mutate_programs=True, mutator=mutator, sexual_mutator=sexual_mutator,
          n_sparse_max=n_sparse_max, return_metrics=True, excl_fs=excl_fs)

step = 0
# how many steps per frame we start with. This gets usually doubled many times
# during the simulation.
# In the article, we usually use 2 or 4 as the starting value, sometimes 1.
steps_per_frame = 2

ku, key = jr.split(key)
programs, env, metrics = run_env(
    ku, programs, env, n_steps, step_f, zoom_sz=6)



def running_average(a, n):
  a = np.concatenate([np.full([n], a[0]), a], axis=0)
  return np.convolve(a, np.ones(n)/n, mode="valid")

if enable_asexual_reproduction:
  plt.plot(running_average(metrics['n_asexual_repr_log'], 100), label="n_asexual_repr_log")
if enable_sexual_reproduction:
  plt.plot(running_average(metrics['n_sexual_repr_log'], 100), label="n_sexual_repr_log")
plt.legend()
plt.show()

plt.plot(running_average(metrics['n_sexes_log'], 100), label="n_sexes_log")
plt.legend()
plt.show()

aid_flat = env.agent_id_grid.flatten()
is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
alive_programs = programs[n_alive_per_id>0]
print("Extracted {} programs.".format(alive_programs.shape[0]))
print("sexes:", vmap(agent_logic.get_sex)(mutator.split_params(alive_programs)[0]))

In [ ]:
# continue...
n_steps = 25000
ku, key = jr.split(key)

programs, env, metrics = run_env(
    ku, programs, env, n_steps, step_f, zoom_sz=6,
    steps_per_frame=64, when_to_double_speed=[],
    curr_asexual_repr=metrics['curr_asexual_repr'],
    curr_sexual_repr=metrics['curr_sexual_repr'])


if enable_asexual_reproduction:
  plt.plot(running_average(metrics['n_asexual_repr_log'], 100), label="n_asexual_repr_log")
if enable_sexual_reproduction:
  plt.plot(running_average(metrics['n_sexual_repr_log'], 100), label="n_sexual_repr_log")
plt.legend()
plt.show()

plt.plot(running_average(metrics['n_sexes_log'], 100), label="n_sexes_log")
plt.legend()
plt.show()

aid_flat = env.agent_id_grid.flatten()
is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
alive_programs = programs[n_alive_per_id>0]
print("Extracted {} programs.".format(alive_programs.shape[0]))
print("sexes:", vmap(agent_logic.get_sex)(mutator.split_params(alive_programs)[0]))
